In [1]:
from ROOT import TFile

def getall(d, basepath="/"):
    "Generator function to recurse into a ROOT file/dir and yield (path, obj) pairs"
    for key in d.GetListOfKeys():
        kname = key.GetName()
        if key.IsFolder():
            # TODO: -> "yield from" in Py3
            for i in getall(d.Get(kname), basepath+kname+"/"):
                yield i
        else:
            yield d.Get(kname)
def get_Bins_from_histo(h):
    bins=[h.GetBinContent(bin_+1) for bin_ in range(h.GetNbinsX())]
    return h.GetTitle() ,  bins


def convert_histos_to_txt(file):
    parent_dir=os.path.dirname(file)
    save_dir=parent_dir.replace(
        os.path.basename(folder_out),
        os.path.basename(os.getcwd())
    )
    !rm -rf {save_dir}/*
    !mkdir -p {save_dir}
    #print(os.path.basename(file))
    f=TFile(file)
    bins=dict(map(get_Bins_from_histo,getall(f)))
    bins["bosons"]=[]
    bosons_list=["w_jets","z_jets","ww","wz","zz"]
    for i in range(len(bins["w_jets"])):
        sum_=0.
        for key in bosons_list:
            sum_+=bins[key][i]
        bins["bosons"].append(sum_)
    bins["signal"]=[]
    signal_list=[]
    for i in range(len(bins["w_jets"])):
        sum_=0.
        for key in bins:
            if not ("LQ_" in key): continue
            signal_list.append(key)
            sum_+=bins[key][i]
        bins["signal"].append(sum_)
    for key in bosons_list+signal_list:
        try: 
            bins.pop(key)
        except:
            continue
    
    for key in bins:
        with open(os.path.join(save_dir,f'{key}.txt'), 'w') as f:
            [f.write(str(bin_)+"\n") for bin_ in bins[key]]



Welcome to JupyROOT 6.22/06


In [2]:
import os
from pathlib import Path
from multiprocessing import Pool
from multiprocessing import cpu_count as n_cpu

folder_out=os.path.join(
    os.path.dirname(os.getcwd()),
    "04_ML_analysis"
)
path_root = Path(folder_out)
files=[root_file.as_posix() for root_file in path_root.glob('**/*.root')]

with Pool(n_cpu()) as p:
    list(p.map(convert_histos_to_txt,files))